# General Imports and Spark Config

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

In [ ]:
config = pyspark.SparkConf().setAll([
    ('spark.executor.memory', '16g'), 
    ('spark.executor.cores', '4'), 
    ('spark.cores.max', '4'),
    ('spark.driver.memory','16'),
    ('spark.executor.instances', '1'),
    ('spark.worker.cleanup.enabled', 'true'),
    ('spark.worker.cleanup.interval', '60'),
    ('spark.worker.cleanup.appDataTtl', '60'),
    ('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.2')
])

Important: In neo4j there is a naming convention, node labels should use camelcase (beginning with uppercase) and relationship labels should use all uppercase with _

Notes from Neo4j regarding the Spark Connector
We recommend individual property fields to be returned, rather than returning graph entity (node, relationship, and path) types. This best maps to Spark’s type system and yields the best results. So instead of writing:

MATCH (p:Person) RETURN p

write the following:

MATCH (p:Person) RETURN id(p) AS id, p.name AS name.

If your query returns a graph entity, use the labels or relationship modes instead.


# Create Spark Session

In [ ]:
spark = SparkSession \
    .builder \
    .config(conf=config) \
    .appName("PatternsInNetwork") \
    .master("spark://172.23.149.212:7077") \
    .getOrCreate()

# Pattern Detection

## Pattern 1: Large Payments in less than a Day

The main goal of this pattern is to detect accounts that have sent a payment transaction which is larger than 100'000 algos (100'000'000'000 Microalgos) and that another account has sent an equal or larger amount further. The results which are returned can be grouped and counted to have an overwiew which addresses made multiple of these calls.

In [ ]:
# for full data query remove the LIMIT 10 phrase

query = """
MATCH (a1:Account)-[r1:PAYMENT]->(a2:Account)-[r2:PAYMENT]->(a3:Account)
USING INDEX r1:PAYMENT(amount)
USING INDEX r2:PAYMENT(amount)
WHERE r2.amount > 100000000000 AND r1.amount > 100000000000 AND a1.account <> a2.account AND a2.account <> a3.account
WITH DISTINCT r1 AS rela, a1.account AS senderAccount LIMIT 10
RETURN senderAccount
"""

queryAllData = """
MATCH (a1:Account)-[r1:PAYMENT]->(a2:Account)-[r2:PAYMENT]->(a3:Account)
USING INDEX r1:PAYMENT(amount)
USING INDEX r2:PAYMENT(amount)
WHERE r2.amount > 100000000000 AND r1.amount > 100000000000 AND a1.account <> a2.account AND a2.account <> a3.account
WITH DISTINCT r1 AS rela, a1.account AS senderAccount
RETURN senderAccount
"""

dfPattern1 = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://172.23.149.212:7687") \
  .option("query", queryAllData) \
  .load() 

Group the data by senderAccount and count how many times the sender has made such a transaction.

In [ ]:
dfPattern1.show()
dfPattern1Grouped = dfPattern1.groupBy("senderAccount").count().sort(col("count").desc())
dfPattern1Grouped.show()

Write results to MongoDB

In [ ]:
dfPattern1Grouped.write.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.212:27017') \
  	.mode("overwrite") \
    .option('spark.mongodb.database', 'algorand_gold') \
  	.option('spark.mongodb.collection', 'Patterns_LargePaymentTransactionAccounts_6') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
  	.save()

## Pattern 2: Accounts that have made many Asset Transfers

The goal is to find accounts that made many asset transfers between each other.

In [ ]:
# for full data query remove the LIMIT 10 phrase

# diejenigen die viel kreieren in asset configuration
# diejenigen finden welche wie oben das gleiche asset weiter transferiert haben in kurzer zeit, mischung aus 1 und 3

query = """
MATCH (a1:Account)-[r:ASSET_TRANSFER]->(a2:Account)
USING INDEX r:ASSET_TRANSFER(transferType)
WHERE a1.account <> a2.account AND r.transferType="transfer"
With a1.account AS senderAccount, count(r) AS rel_count
RETURN senderAccount, rel_count
ORDER BY rel_count DESC 
"""

queryAllData = """
MATCH (a1:Account)-[r:ASSET_TRANSFER]->(a2:Account)
USING INDEX r:ASSET_TRANSFER(transferType)
WHERE a1.account <> a2.account AND r.transferType="transfer"
WITH a1.account AS senderAccount, count(r) AS rel_count
RETURN senderAccount, rel_count
ORDER BY rel_count DESC 
"""

dfPattern2 = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://172.23.149.212:7687") \
  .option("query", queryAllData) \
  .load()

In [ ]:
alternativeQuery = """
MATCH (a1:Account)-[r:ASSET_TRANSFER]->(a2:Account) 
WHERE a1.account <> a2.account 
WITH size((a1:Account)-[r:ASSET_TRANSFER]->(a2:Account) ) AS rel_count, a1.account AS senderAccount, a2.account as rcv LIMIT 10
RETURN senderAccount, rcv, rel_count
ORDER BY rel_count DESC
"""

In [ ]:
dfPattern2.show()

In [ ]:
dfPattern2.write.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.212:27017') \
  	.mode("overwrite") \
    .option('spark.mongodb.database', 'algorand_gold') \
  	.option('spark.mongodb.collection', 'Patterns_ManyAssetTransferAcc_6') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
  	.save()

## Pattern 3: Accounts that have created a lot of assets

The goal is to find accounts that have sent many asset_configuration transactions to create an asset.

In [ ]:
# for full data query remove the LIMIT 10 phrase

query = """
MATCH (a1:Account)-[r:ASSET_CONFIGURATION]->(a2:Asset)
WHERE r.configurationType = "creation"
WITH a1.account AS senderAccount, COUNT(r) AS rel_count LIMIT 10
RETURN senderAccount, rel_count
ORDER BY rel_count DESC
"""

queryAllData = """
MATCH (a1:Account)-[r:ASSET_CONFIGURATION]->(a2:Asset)
WHERE r.configurationType = "creation"
WITH a1.account AS senderAccount, count(r) AS rel_count
RETURN senderAccount, rel_count
ORDER BY rel_count DESC
"""

dfPattern3 = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://172.23.149.212:7687") \
  .option("query", queryAllData) \
  .load() 

In [ ]:
dfPattern3.show()

In [ ]:
dfPattern3.write.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.212:27017') \
  	.mode("overwrite") \
    .option('spark.mongodb.database', 'algorand_gold') \
  	.option('spark.mongodb.collection', 'Patterns_AccAssetCreation_6') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
  	.save()

## Pattern 4: SmartContract calls where other people have called the same SC shortly after

The goal of this pattern is to detect smart contracts and people that have all sent transactions to the same SC in a short amount of time.

In [ ]:
# for full data query remove the LIMIT 10 phrase

query = """
MATCH (a1:Account)-[r1:APPLICATION_CALL]->(app:Application)<-[r2:APPLICATION_CALL]-(a2:Account)
USING INDEX r1:APPLICATION_CALL(blockNumber)
USING INDEX r2:APPLICATION_CALL(blockNumber)
WHERE a1.account <> a2.account AND r1.blockNumber > 0 AND r2.blockNumber > 0 AND abs(r2.blockNumber - r1.blockNumber) < 17280 
WITH a1.account AS account, app.application AS application, r1.blockNumber as blockNumber LIMIT 100 
RETURN DISTINCT application, account, blockNumber
"""

queryAllData = """
MATCH (a1:Account)-[r1:APPLICATION_CALL]->(app:Application)<-[r2:APPLICATION_CALL]-(a2:Account)
USING INDEX r1:APPLICATION_CALL(blockNumber)
USING INDEX r2:APPLICATION_CALL(blockNumber)
WHERE a1.account <> a2.account AND r1.blockNumber > 0 AND r2.blockNumber > 0 AND abs(r2.blockNumber - r1.blockNumber) < 17280 
WITH a1.account AS account, app.application AS application, r1.blockNumber as blockNumber
RETURN DISTINCT application, account, blockNumber
"""

dfPattern4 = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://172.23.149.212:7687") \
  .option("query", queryAllData) \
  .load()

In [ ]:
dfPattern4.show()

Write the results to MongoDB silver collection.

In [ ]:
dfPattern4.write.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.212:27017') \
  	.mode("overwrite") \
    .option('spark.mongodb.database', 'algorand_silver') \
  	.option('spark.mongodb.collection', 'Patterns_ScCallsFromDifferentAccRaw_6') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
  	.save()

Group and count the applications.

In [ ]:
dfPattern4Grouped = dfPattern4.groupBy("application").count()

In [ ]:
dfPattern4Grouped.show()

Write grouped results to MongoDB

In [ ]:
dfPattern4Grouped.write.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.212:27017') \
  	.mode("overwrite") \
    .option('spark.mongodb.database', 'algorand_gold') \
  	.option('spark.mongodb.collection', 'Patterns_ScCallsGrouped_6') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
  	.save()

## Pattern 5: Accounts that had a lot small of transfers between each other

The goal is to find accounts that made more than 100 transactions between each other with an amount smaller than 0.1 Algos.

In [ ]:
# for full data query remove the LIMIT 10 phrase

query = """
MATCH (a1:Account)-[r:PAYMENT]->(a2:Account) 
WHERE r.amount < 100000 AND a1.account <> a2.account 
WITH count(r) AS rel_count, a1.account AS senderAccount, a2.account AS receiverAccount LIMIT 10 
WHERE rel_count > 100 
RETURN senderAccount, receiverAccount, rel_count
ORDER BY rel_count DESC
"""

queryAllData = """
MATCH (a1:Account)-[r:PAYMENT]->(a2:Account) 
WHERE r.amount < 100000 AND a1.account <> a2.account 
WITH count(r) AS rel_count, a1.account AS senderAccount, a2.account AS receiverAccount
WHERE rel_count > 100 
RETURN senderAccount, receiverAccount, rel_count
ORDER BY rel_count DESC
"""

dfPattern5 = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://172.23.149.212:7687") \
  .option("query", queryAllData) \
  .load()

In [ ]:
dfPattern5.show()

Save the results in MongoDB

In [ ]:
dfPattern5.write.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.212:27017') \
  	.mode("overwrite") \
    .option('spark.mongodb.database', 'algorand_gold') \
  	.option('spark.mongodb.collection', 'Patterns_AccountsWithManyPaymentTransactions_6') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
  	.save()

Sum over all sender accounts to see how many of these transactions were made by each account

In [ ]:
dfPattern5Summed = dfPattern5.groupBy("senderAccount").sum("rel_count")

In [ ]:
dfPattern5Summed.show()

In [ ]:
dfPattern5Summed.write.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.212:27017') \
  	.mode("overwrite") \
    .option('spark.mongodb.database', 'algorand_gold') \
  	.option('spark.mongodb.collection', 'Patterns_AccWithManyPaymentTransactionsSum_6') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
  	.save()

# Patterns that make use of GraphAlgorithms

Create the graphs projections that are needed for the algorithms.

In [ ]:
query = """
CALL gds.graph.project(
  "paymentGraph",
  "Account",                         
  {
    PAYMENT: {properties: ["blockNumber", "amount"]}
  }           
)
 YIELD
  graphName AS graph, nodeProjection, nodeCount AS nodes, relationshipProjection, relationshipCount AS rels
 RETURN graph, nodeProjection, nodes, relationshipProjection, rels
"""


dfPaymentGraphProjection = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://172.23.149.212:7687") \
  .option("query", query) \
  .option("partitions", "1") \
  .load()

## Pattern 6: Degree Centrality in Payment Subnetwork

The goal of this pattern is to detect degree centralities in asset transfer senders. We search for the 50 nodes with the highest degrees to detect the most important accounts.

In [ ]:
query = """
CALL gds.degree.stream('paymentGraph')
YIELD nodeId, score
WITH gds.util.asNode(nodeId).account AS account, score AS degree
ORDER BY degree DESC LIMIT 50
RETURN account, degree
"""

dfPattern6 = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://172.23.149.212:7687") \
  .option("query", query) \
  .option("partitions", "1") \
  .load()

In [ ]:
dfPattern6.show()

In [ ]:
dfPattern6.write.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.212:27017') \
  	.mode("overwrite") \
    .option('spark.mongodb.database', 'algorand_gold') \
  	.option('spark.mongodb.collection', 'Patterns_DegreeCentrality_Top50_6') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
  	.save()

## Pattern 7: Eigenvector Centrality in Payment Subnetwork

The goal of this pattern is to detect centralities in asset transfer senders.

In [ ]:
query = """
CALL gds.eigenvector.stream('paymentGraph')
YIELD nodeId, score
WITH gds.util.asNode(nodeId).account AS account, score AS eigenVectorScore
ORDER BY eigenVectorScore DESC LIMIT 10
RETURN account, eigenVectorScore
"""

dfPattern7 = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://172.23.149.212:7687") \
  .option("query", query) \
  .option("partitions", "1") \
  .load()

In [ ]:
dfPattern7.show()

In [ ]:
dfPattern7.write.format("mongodb") \
	.option('spark.mongodb.connection.uri', 'mongodb://172.23.149.212:27017') \
  	.mode("overwrite") \
    .option('spark.mongodb.database', 'algorand_gold') \
  	.option('spark.mongodb.collection', 'Patterns_EigenvectorCentrality_Top10_6') \
  	.option("forceDeleteTempCheckpointLocation", "true") \
  	.save()

# Stopping Spark Context

Stopping context and removing the graph projection

In [ ]:
query = """
CALL gds.graph.drop('paymentGraph') 
YIELD graphName 
RETURN graphName
"""

dfPaymentGraphProjection = spark.read.format("org.neo4j.spark.DataSource") \
  .option("url", "bolt://172.23.149.212:7687") \
  .option("query", query) \
  .load()

In [ ]:
spark.stop()